In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time
import ipywidgets as widgets
from ipywidgets import interact, interactive
from collections import defaultdict
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import show, output_notebook, push_notebook, curdoc
from bokeh.models import CustomJS, Slider, RangeSlider, CheckboxGroup
from bokeh.layouts import column, row

In [2]:
estates = pd.read_csv("Sacramentorealestatetransactions.csv")
estates["type"] = estates["type"].astype('category')
estates["sale_date"] = estates["sale_date"].astype('category')
estates["type"] = estates["type"].astype("category")
estates["sale_date"] = estates["sale_date"].astype("category")
estates["cat_beds"]=estates["beds"].astype("category")
estates["cat_baths"]=estates["baths"].astype("category")
estates_cat = estates[["type", "sale_date", "cat_beds", "cat_baths"]]
estates["cat_zip"] = estates["zip"].astype("category")
estates_cat["zip"] = estates["cat_zip"]

C:\Users\Belladonna Melania\AppData\Local\Temp\ipykernel_2732\1380871667.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estates_cat["zip"] = estates["cat_zip"]


In [3]:
estates.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879,2,1,95838
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028,3,1,95823
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839,2,1,95815
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146,2,1,95815
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768,2,1,95824


In [4]:
estates.groupby("type").count()

,street,city,zip,state,beds,baths,sq__ft,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
type,,,,,,,,,,,,,,
Condo,54,54,54,54,54,54,54,54,54,54,54,54,54,54
Multi-Family,13,13,13,13,13,13,13,13,13,13,13,13,13,13
Residential,917,917,917,917,917,917,917,917,917,917,917,917,917,917
Unkown,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [5]:
estates.groupby("city").count().sort_values(by="beds")

,street,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
city,,,,,,,,,,,,,,
SHINGLE SPRINGS,1,1,1,1,1,1,1,1,1,1,1,1,1,1
GREENWOOD,1,1,1,1,1,1,1,1,1,1,1,1,1,1
FORESTHILL,1,1,1,1,1,1,1,1,1,1,1,1,1,1
MATHER,1,1,1,1,1,1,1,1,1,1,1,1,1,1
SLOUGHHOUSE,1,1,1,1,1,1,1,1,1,1,1,1,1,1
GARDEN VALLEY,1,1,1,1,1,1,1,1,1,1,1,1,1,1
MEADOW VISTA,1,1,1,1,1,1,1,1,1,1,1,1,1,1
COOL,1,1,1,1,1,1,1,1,1,1,1,1,1,1
WALNUT GROVE,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [6]:
estates[estates["city"] == "ELK GROVE"].groupby(["beds"]).count()

,street,city,zip,state,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
beds,,,,,,,,,,,,,,
0,7,7,7,7,7,7,7,7,7,7,7,7,7,7
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,8,8,8,8,8,8,8,8,8,8,8,8,8,8
3,39,39,39,39,39,39,39,39,39,39,39,39,39,39
4,43,43,43,43,43,43,43,43,43,43,43,43,43,43
5,14,14,14,14,14,14,14,14,14,14,14,14,14,14
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [7]:
estates[estates["city"] == "ELK GROVE"].groupby(estates["price"] >= 250000).count()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
price,,,,,,,,,,,,,,,
False,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55
True,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59


In [8]:
estates[(estates["city"] == "ELK GROVE")].groupby(estates["baths"]).count()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
baths,,,,,,,,,,,,,,,
0,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7
1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
3,29,29,29,29,29,29,29,29,29,29,29,29,29,29,29
4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [9]:
estates[(estates["city"] == "ELK GROVE") & (estates["type"] == "Residential")].count()

street       108
city         108
zip          108
state        108
beds         108
baths        108
sq__ft       108
type         108
sale_date    108
price        108
latitude     108
longitude    108
cat_beds     108
cat_baths    108
cat_zip      108
dtype: int64

In [10]:
estates[(estates["city"] == "ELK GROVE")].count()

street       114
city         114
zip          114
state        114
beds         114
baths        114
sq__ft       114
type         114
sale_date    114
price        114
latitude     114
longitude    114
cat_beds     114
cat_baths    114
cat_zip      114
dtype: int64

In [11]:
estates[(estates["city"] == "ELK GROVE") & (estates["type"] == "Residential")].count()

street       108
city         108
zip          108
state        108
beds         108
baths        108
sq__ft       108
type         108
sale_date    108
price        108
latitude     108
longitude    108
cat_beds     108
cat_baths    108
cat_zip      108
dtype: int64

In [12]:
estates[(estates["city"] == "ELK GROVE")].max()

C:\Users\Belladonna Melania\AppData\Local\Temp\ipykernel_2732\1771599876.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  estates[(estates["city"] == "ELK GROVE")].max()


street       9967 HATHERTON WAY
city                  ELK GROVE
zip                       95758
state                        CA
beds                          6
baths                         4
sq__ft                     4303
price                    510000
latitude              38.452075
longitude           -121.335541
dtype: object

In [13]:
estates[(estates["city"] == "ELK GROVE")].min()

C:\Users\Belladonna Melania\AppData\Local\Temp\ipykernel_2732\2711817014.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  estates[(estates["city"] == "ELK GROVE")].min()


street       10085 ATKINS DR
city               ELK GROVE
zip                    95624
state                     CA
beds                       0
baths                      0
sq__ft                     0
price                  71000
latitude             38.3052
longitude        -121.484325
dtype: object

In [14]:
estates[(estates["city"] == "ELK GROVE") & 
        (estates["type"] == "Residential") &
        (estates["beds"] >= 3) & 
        (estates["beds"] <= 5) & 
        (estates["baths"] >= 2) &
        (estates["baths"] <= 3) &
        (estates["price"] >= 130000) & 
        (estates["price"] <= 500000) &
        (estates["sq__ft"] >= 2230) & 
        (estates["sq__ft"] <= 4500) 
       ].mean()

C:\Users\Belladonna Melania\AppData\Local\Temp\ipykernel_2732\343898925.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  ].mean()


zip           95727.161290
beds              4.225806
baths             2.741935
sq__ft         2794.967742
price        345465.322581
latitude         38.409051
longitude      -121.418294
dtype: float64

In [15]:
estates.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879,2,1,95838
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028,3,1,95823
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839,2,1,95815
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146,2,1,95815
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768,2,1,95824


In [21]:
estates[(estates["city"] == "ELK GROVE")].groupby("price").count()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,latitude,longitude,cat_beds,cat_baths,cat_zip
price,,,,,,,,,,,,,,
71000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
98000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
100000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
133000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
141000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
484500,1,1,1,1,1,1,1,1,1,1,1,1,1,1
489332,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [54]:
estates[(estates["city"] == "LINCOLN") & (estates["price"] <= 4897)].sort_values("street")

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
571,100 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.886091,-121.289744,0,0,95648
588,100 CRYSTALWOOD WAY,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.886282,-121.289706,0,0,95648
587,108 CRYSTALWOOD WAY,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.886282,-121.289646,0,0,95648
574,11 E ST,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.884879,-121.290257,0,0,95648
586,116 CRYSTALWOOD WAY,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.886282,-121.289586,0,0,95648
599,15 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.885571,-121.289421,0,0,95648
598,19 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.885376,-121.289414,0,0,95648
575,19 E ST,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.885017,-121.290262,0,0,95648
554,20 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.885327,-121.289412,0,0,95648
597,23 CRYSTALWOOD CIR,LINCOLN,95648,CA,0,0,0,Residential,Mon May 19 00:00:00 EDT 2008,4897,38.885181,-121.289406,0,0,95648


In [53]:
estates[(estates["city"] == "LINCOLN") & (estates["price"] <= 4897)].groupby("beds").count()

,street,city,zip,state,baths,sq__ft,type,sale_date,price,latitude,longitude,cat_beds,cat_baths,cat_zip
beds,,,,,,,,,,,,,,
0,46,46,46,46,46,46,46,46,46,46,46,46,46,46
3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1
